In [2]:
from pyspark.sql import SparkSession 
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [3]:
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [4]:
training.show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
|   Jon| 47|        26|250000|
|Zeinab| 35|         9|165000|
|Sheena| 41|        19|150000|
| Tanya| 30|         7|120000|
|  Alex| 35|        11|180000|
|Daljit| 33|        10|100000|
+------+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [8]:
from pyspark.ml.feature import VectorAssembler 
featureassembler=VectorAssembler(inputCols=["Age", "Experience"],outputCol="Independent features")

In [9]:
output=featureassembler.transform(training)

In [10]:
output.show()

+------+---+----------+------+--------------------+
|  Name|Age|Experience|Salary|Independent features|
+------+---+----------+------+--------------------+
|   Jon| 47|        26|250000|         [47.0,26.0]|
|Zeinab| 35|         9|165000|          [35.0,9.0]|
|Sheena| 41|        19|150000|         [41.0,19.0]|
| Tanya| 30|         7|120000|          [30.0,7.0]|
|  Alex| 35|        11|180000|         [35.0,11.0]|
|Daljit| 33|        10|100000|         [33.0,10.0]|
+------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent features']

In [13]:
finalised_data=output.select("Independent features", "Salary")

In [14]:
finalised_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [47.0,26.0]|250000|
|          [35.0,9.0]|165000|
|         [41.0,19.0]|150000|
|          [30.0,7.0]|120000|
|         [35.0,11.0]|180000|
|         [33.0,10.0]|100000|
+--------------------+------+



In [16]:
from pyspark.ml.regression import LinearRegression 
train_data,test_data=finalised_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor=regressor.fit(train_data)

24/04/06 18:02:02 WARN Instrumentation: [e9fbf60e] regParam is zero, which might cause numerical instability and overfitting.


In [17]:
regressor.coefficients

DenseVector([31780.1047, -21544.5026])

In [18]:
regressor.intercept

-707670.1570672665

In [19]:
pred_results=regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent features|Salary|       prediction|
+--------------------+------+-----------------+
|          [35.0,9.0]|165000|210732.9842931059|
+--------------------+------+-----------------+



In [21]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(45732.984293105896, 2091505852.3534706)